In [1]:
import os.path as op
import os
import numpy as np
import pandas as pd
from fpdf import FPDF
import shutil

import mne
import matplotlib.pyplot as plt
import mne_bids

import sys

import argparse
import itertools
import json
import pandas as pd
import pickle


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/traitlets/config/application.py",

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/headnode1/abry4213/.conda/envs/pyspi/lib/python3.9/site-packages/traitlets/config/application.py",

AttributeError: _ARRAY_API not found

In [3]:
subject_id="CB003"
visit_id="1"
bids_root="/headnode1/abry4213/data/Cogitate_Batch2/MEG_Data/"

# Set path to preprocessing derivatives
prep_deriv_root = op.join(bids_root, "derivatives", "preprocessing")
prep_figure_root =  op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "figures")
prep_report_root =  op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "reports")
prep_code_root = op.join(prep_deriv_root,
                            f"sub-{subject_id}",f"ses-{visit_id}","meg",
                            "codes")

# Lopp over runs
data_path = os.path.join(bids_root,f"sub-{subject_id}",f"ses-{visit_id}","meg")
run = "05"
bids_task = 'dur'

In [5]:
region_option="hypothesis_driven"
factor = ['Category', 'Task_relevance', "Duration"]
conditions = [['face', 'object', 'letter', 'false'],
              ['Relevant non-target', 'Irrelevant'],
              ['1000ms']]

fs_deriv_root = op.join(bids_root, "derivatives", "fs")
rois_deriv_root = op.join(bids_root, "derivatives", "roilabel")
prep_deriv_root = op.join(bids_root, "derivatives", "preprocessing")

time_series_output_path = op.join(bids_root, "derivatives", "MEG_time_series")
if not op.exists(time_series_output_path):
    os.makedirs(time_series_output_path, exist_ok=True)

# Time series output path for this subject
subject_time_series_output_path = op.join(time_series_output_path, f"sub-{subject_id}", f"ses-{visit_id}", "meg")
if not op.exists(subject_time_series_output_path):
    os.makedirs(subject_time_series_output_path, exist_ok=True)
    
# Set task
bids_task = 'dur'

# Read epoched data
bids_path_epo = mne_bids.BIDSPath(
    root=prep_deriv_root, 
    subject=subject_id,  
    datatype='meg',  
    task=bids_task,
    session=visit_id, 
    suffix='epo',
    extension='.fif',
    check=False)


In [6]:

# Helper function to create a dictionary of ROI labels depending on the type of region subset requested
def compute_ROI_labels(labels_atlas, region_option, rois_deriv_root):
    # Create dictionary to store labels and vertices
    labels_dict = {}
    if region_option == 'hypothesis_driven':
        # Read GNW and IIT ROI list
        f = open(op.join(rois_deriv_root,
                        'hypothesis_driven_ROIs.json'))
        hypothesis_driven_ROIs = json.load(f)

        # GNWT ROIs
        print("GNWT ROIs:")
        for lab in hypothesis_driven_ROIs['GNWT_ROIs']:
            print(lab)
            labels_dict["GNWT_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # IIT ROIs
        print("IIT ROIs")
        for lab in hypothesis_driven_ROIs['IIT_ROIs']:
            print(lab)
            labels_dict["IIT_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # Category-selective ROIs
        print("Category-selective ROIs:")
        for lab in hypothesis_driven_ROIs['Category_Selective_ROIs']:
            print(lab)
            labels_dict["Category_Selective_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # Merge all labels in a single one separatelly for GNW and IIT 
        labels_dict['GNWT_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'GNWT' in l_name])
        labels_dict['IIT_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'IIT' in l_name])
        labels_dict['Category_Selective_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'Category_Selective' in l_name])

        # Only keep the meta-ROIs
        labels_dict = {k: v for k, v in labels_dict.items() if 'meta_ROI' in k}

    else:
        for label in labels_atlas: 
            label_name = label.name
            labels_dict[label_name] =  np.sum([label])

    return labels_dict

# Helper function to compute covariance matrices and inverse solution 
def fit_cov_and_inverse(subject_id, visit_id, factor, conditions, bids_root, downsample=True, tmin=-0.5, tmax=1.99):
    # Set directory paths
    prep_deriv_root = op.join(bids_root, "derivatives", "preprocessing")
    fwd_deriv_root = op.join(bids_root, "derivatives", "forward")
    source_deriv_root = op.join(bids_root, "derivatives", "source_dur_ERF")

    if not op.exists(source_deriv_root):
        os.makedirs(source_deriv_root, exist_ok=True)

    source_figure_root =  op.join(source_deriv_root,
                                f"sub-{subject_id}",f"ses-{visit_id}","meg",
                                "figures")
    if not op.exists(source_figure_root):
        os.makedirs(source_figure_root)

    # Set task
    bids_task = 'dur'
    
    # Read epoched data
    bids_path_epo = mne_bids.BIDSPath(
        root=prep_deriv_root, 
        subject=subject_id,  
        datatype='meg',  
        task=bids_task,
        session=visit_id, 
        suffix='epo',
        extension='.fif',
        check=False)
    
    bids_path_epo_rs = mne_bids.BIDSPath(
        root=prep_deriv_root, 
        subject=subject_id,  
        datatype='meg',  
        task=bids_task,
        session=visit_id, 
        suffix='epo_rs',
        extension='.fif',
        check=False)
    
    print("Loading epochs data")
    epochs_all = mne.read_epochs(bids_path_epo.fpath, preload=True)

    if not downsample:
        epochs_final = epochs_all

    # If downsampling is requested
    else:
        print("Applying downsampling")
        if os.path.exists(bids_path_epo_rs.fpath):
            epochs_rs = mne.read_epochs(bids_path_epo_rs.fpath,
                                    preload=True)
        else:
            epochs_all = mne.read_epochs(bids_path_epo.fpath,
                                    preload=True)
            resample_epochs(epochs_all, sfreq, bids_path_epo_rs, tmin=tmin, tmax=tmax)
            epochs_rs = mne.read_epochs(bids_path_epo_rs.fpath, preload=True)
        epochs_final = epochs_rs

    # Run baseline correction
    print("Running baseline correction")
    b_tmin = tmin
    b_tmax = 0.
    baseline = (b_tmin, b_tmax)
    epochs_final.apply_baseline(baseline=baseline)

    # Compute rank
    print("Computing the rank")
    if os.path.isfile(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_rank.pkl"):
        with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_rank.pkl", 'rb') as f:
            rank = pickle.load(f)
    else: 
        rank = mne.compute_rank(epochs_final, 
                                tol=1e-6, 
                                tol_kind='relative')
        with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_rank.pkl", 'wb') as f:
            pickle.dump(rank, f)

    # Read forward model
    print("Reading forward model")
    bids_path_fwd = bids_path_epo.copy().update(
            root=fwd_deriv_root,
            task=bids_task,
            suffix="surface_fwd",
            extension='.fif',
            check=False)
    fwd = mne.read_forward_solution(bids_path_fwd.fpath)

    # Compute covariance matrices
    print("Computing covariance matrices")
    if os.path.isfile(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_common_cov.pkl"): 
        with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_common_cov.pkl", 'rb') as f:
            common_cov = pickle.load(f)
    else:
        base_cov = mne.compute_covariance(epochs_final, 
                                        tmin=-0.5, 
                                        tmax=0, 
                                        n_jobs=n_jobs,
                                        method='empirical', 
                                        rank=rank)

        active_cov = mne.compute_covariance(epochs_final, 
                                        tmin=0,
                                        tmax=None,
                                        n_jobs=n_jobs,
                                        method='empirical', 
                                        rank=rank)
        common_cov = base_cov + active_cov

        with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_common_cov.pkl", 'wb') as f:
            pickle.dump(common_cov, f)

    # Make inverse operator
    inverse_operator = mne.minimum_norm.make_inverse_operator(
        epochs_final.info,
        fwd, 
        common_cov,
        loose=.2,
        depth=.8,
        fixed=False,
        rank=rank,
        use_cps=True)

    # Find all combinations between variables' levels
    if len(factor) == 1:
        cond_combs = list(itertools.product(conditions[0]))
    if len(factor) == 2:
        cond_combs = list(itertools.product(conditions[0],
                                            conditions[1]))
    if len(factor) == 3:
        cond_combs = list(itertools.product(conditions[0],
                                            conditions[1],
                                            conditions[2]))
        
    return epochs_final, inverse_operator, cond_combs


In [11]:
# Use Desikan--Killiany atlas to compute dictionary of labels
labels_atlas = mne.read_labels_from_annot(
    "sub-"+subject_id, 
    parc='aparc.a2009s',
    subjects_dir=fs_deriv_root)
labels_dict = compute_ROI_labels(labels_atlas, region_option, rois_deriv_root)

# Find epochs_rs, inverse_operator, cond_combs
print("Now finding inverse operator")
epochs_final, inverse_operator, cond_combs = fit_cov_and_inverse(subject_id, visit_id, factor, conditions, bids_root, downsample=False)

# extract time course in label with pca_flip mode
src = inverse_operator['src']


Reading labels from parcellation...
   read 75 labels from /headnode1/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/fs/sub-CB003/label/lh.aparc.a2009s.annot
   read 75 labels from /headnode1/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/fs/sub-CB003/label/rh.aparc.a2009s.annot
GNWT ROIs:
G_and_S_cingul-Ant
G_and_S_cingul-Mid-Ant
G_and_S_cingul-Mid-Post
G_front_inf-Opercular
G_front_inf-Orbital
G_front_inf-Triangul
G_front_middle
Lat_Fis-ant-Horizont
Lat_Fis-ant-Vertical
S_front_inf
S_front_middle
S_front_sup
IIT ROIs
G_cuneus
G_occipital_sup
G_oc-temp_med-Lingual
G_oc-temp_med-Parahip
G_temporal_inf
Pole_occipital
Pole_temporal
S_calcarine
S_intrapariet_and_P_trans
S_oc_sup_and_transversal
S_temporal_sup
Category-selective ROIs:
G_and_S_occipital_inf
G_oc-temp_lat-fusifor
G_occipital_middle
S_oc_middle_and_Lunatus
Now finding inverse operator
Loading epochs data
Reading /headnode1/abry4213/data/Cogitate_Batch2/MEG_Data/derivatives/preprocessing/sub-CB003/ses-1/meg/sub-CB003_s